# Linear Algebra - cont'd

In [2]:
import numpy as np
import scipy.linalg as la

### Iterative methods:

In [4]:
def jacobi_method(A, b, max_it=500, eps=0.0):
    """
    Solves system of linear equations iteratively using Jacobi's algorithm.
    Args:
        A (array_like): A n-by-n regular matrix
        b (array_like): RHS vector of size n
        max_it (int): Maximum number of iterations
        eps (float): Error tolerance
    Returns:
        numpy.ndarray: Vector of solution
    """
    n, n = A.shape
    x = np.zeros(n)
    x_new = np.zeros(n)   
    for k in range(max_it):
        for i in range(n):
            x_new[i] = (1.0 / A[i,i]) * (b[i] - np.dot(A[i,:i], x[:i]) - np.dot(A[i,i+1:], x[i+1:]))           
        x = x_new
        if(la.norm(np.dot(A, x) - b) < eps):
            break   
    return x

In [5]:
# test of the jacobi_method function
A = np.random.rand(3, 3) + 10. * np.eye(3) # create diagonally dominant matrix to ensure convergence
b = np.random.rand(3)
print(jacobi_method(A, b, 20))
print(la.solve(A, b))

[0.03442613 0.08663328 0.02184537]
[0.03442613 0.08663328 0.02184537]


In [6]:
def gauss_seidel_method(A, b, max_it=500, eps=0.0):
    """
    Solves system of linear equations iteratively using Gauss-Seidel's algorithm.
    Args:
        A (array_like): A n-by-n regular matrix
        b (array_like): RHS vector of size n
        max_it (int): Maximum number of iterations
        eps (float): Error tolerance
    Returns:
        numpy.ndarray: Vector of solution
    """
    n, n = A.shape
    x = np.zeros(n)
    x_new = np.zeros(n)  
    for k in range(max_it):
        for i in range(n):
            x_new[i] = (1.0 / A[i,i]) * (b[i] - np.dot(A[i,:i], x_new[:i]) - np.dot(A[i,i+1:], x[i+1:]))      
        x = x_new
        if(la.norm(np.dot(A, x) - b) < eps):
            break  
    return x

In [7]:
# test of the gauss_seidel_method function
A = np.random.rand(3, 3) + 10. * np.eye(3) # create diagonally dominant matrix to ensure convergence
b = np.random.rand(3)
print(gauss_seidel_method(A, b, 20))
print(la.solve(A, b))

[0.04673846 0.03862034 0.0595774 ]
[0.04673846 0.03862034 0.0595774 ]


In [8]:
def successive_overrelaxation_method(A, b, omega=1.0, max_it=500, eps=0.0):
    """
    Solves system of linear equations iteratively using successive overrelaxation (SOR) method.
    Args:
        A (array_like): A n-by-n regular matrix
        b (array_like): RHS vector of size n
        max_it (int): Maximum number of iterations
        eps (float): Error tolerance
    Returns:
        numpy.ndarray: Vector of solution
    """
    n, n = A.shape
    x = np.zeros(n)
    x_new = np.zeros(n)
    for k in range(max_it):
        for i in range(n):
            x_new[i] = (1.0 / A[i,i]) * (b[i] - np.dot(A[i,:i], x_new[:i]) - np.dot(A[i,i+1:], x[i+1:]))
        x += omega * (x_new - x)
        if(la.norm(np.dot(A, x) - b) < eps):
            break
    return x

In [9]:
# test of the successive_overrelaxation_method function
A = np.random.rand(3, 3) + 10. * np.eye(3) # create diagonally dominant matrix to ensure convergence
b = np.random.rand(3)

L = np.tril(A, -1) # get lower triangular matrix with zeros on diagonal
U = np.triu(A, 1) # get upper triangular matrix with zeros on diagonal
D = A - L - U # get diagonal matrix
B = -np.dot(la.inv(D + L), U) # calculate iteration matrix
rho = np.max(np.abs(la.eigvals(B))) # find spectral radius (i.e. maximal eigenvalue in absolute value)
omega = 2.0 / (1.0 + np.sqrt(1.0 - rho**2)) # find optimal relaxation factor

print(successive_overrelaxation_method(A, b, omega, 20))
print(la.solve(A, b))

[0.09103014 0.01126088 0.00920895]
[0.09103014 0.01126088 0.00920895]


In [10]:
def power_iteration(A, max_it=500):
    """
    Finds the greatest eigen value (in absolute value) of given matrix and its corresponding eigenvector.
    Args:
        A (array_like): A n-by-n diagonalizable matrix
        max_it (int): Maximum number of iterations
    Returns:
        numpy.ndarray: Eigenvector corresponding to a greatest eigenvalue (in absolute value)
        float: Greatest eigenvalue (in absolute value)
    """
    n, n = A.shape
    eigen_vec = np.ones(n)
    for i in range(max_it):
        eigen_vec_new = np.dot(A, eigen_vec)
        eigen_vec = eigen_vec_new
    eigen_vec = eigen_vec / la.norm(eigen_vec_new)
    eigen_val = la.norm(np.dot(A, eigen_vec))
    return eigen_vec, eigen_val

In [11]:
# test of the power_iteration function
A = np.random.rand(3, 3)
e_vec, e_val = power_iteration(A, 20)
print(e_val)
print(np.max(np.abs(la.eigvals(A))))

1.5675451676482435
1.567545167648242


### Gradient methods:

In [12]:
def conjugate_gradient_method(A, b, x, eps=1.0e-10):
    """
    Solves system of linear equations using conjugate gradient method.
    Args:
        A (array_like): A n-by-n regular matrix
        b (array_like): RHS vector of size n
        x (array_like): Initial guess
        eps (float): Error tolerance
    Returns:
        numpy.ndarray: Vector of solution
    """
    r = b - np.dot(A, x)
    p = r
    rs_old = np.dot(r.T, r)
    for i in range(len(b)):
        Ap = np.dot(A, p)
        alpha = rs_old / (np.dot(p.T, Ap))
        x = x + np.dot(alpha, p)
        r = r - np.dot(alpha, Ap)
        rs_new = np.dot(r.T, r)
        if np.sqrt(rs_new) < eps:
            break
        p = r + np.dot((rs_new / rs_old), p);
        rs_old = rs_new;
    return x

In [13]:
# test of the conjugate_gradient_method function

n = 3 # size of the problem
A = np.random.rand(n, n)
A = np.tril(A) + np.tril(A, -1).T # generate random symmetric matrix (should check also wheter the matrix is positive-definite)
b = np.random.rand(n) # random RHS vector
x_0 = np.zeros(len(b)) # initial guess

print(conjugate_gradient_method(A, b, x_0))
print(la.solve(A, b))

[0.04844102 0.73947705 0.03438916]
[0.04844102 0.73947705 0.03438916]
